### Reference

In [58]:
"""
@misc{MobilNet_SSD_opencv,
  title={MobilNet_SSD for object detection on Keras and TensorFlow},
  author={djmv},
  year={2018},
  publisher={Github},
  journal={GitHub repository},
  howpublished={\url{https://github.com/djmv/MobilNet_SSD_opencv.git}},
  }
"""


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 198-199: truncated \uXXXX escape (<ipython-input-58-afb869174712>, line 10)

### Loading libraries

In [1]:

#Import the neccesary libraries
import numpy as np
import argparse
import cv2 
from gtts import gTTS
import os
import speech_recognition as sr
from playsound import playsound


### Loading model

In [2]:
# Labels of Network.
classNames = { 'background':0,
    'aeroplane':1, 'bicycle':2 , 'bird': 3, 'boat':4,
    'bottle':5, 'bus':6, 'car':7, 'cat':8, 'chair':9,
    'cow':10, 'diningtable':11,  'dog':12,  'horse':13,
    'motorbike':14, 'person':15, 'pottedplant':16,
    'sheep':17, 'sofa':18, 'train':20,  'tvmonitor':20 }
    

#Load the Caffe model 
net = cv2.dnn.readNetFromCaffe('MobileNetSSD_deploy.prototxt', 'MobileNetSSD_deploy.caffemodel')


### Camera

In [14]:
cascPath = sys.argv[1]
faceCascade = cv2.CascadeClassifier(cascPath)

cap= cv2.VideoCapture(0)

## Object to Detect

### Voice to text and Voice assistance 

# Initialize recognizer class (for recognizing the speech)
r = sr.Recognizer()

# Reading Microphone as source
# listening the speech and store in audio_text variable
def talk():
    with sr.Microphone() as source:
        what="what are you loking for, give me one word"
        file = "what.mp3"
        # initialize tts, create mp3 and play
        #tts = gTTS(what)
        #tts.save(file)
        os.system("mpg123 " + file)
        playsound(file)
        
        audio_text = r.listen(source)
        print("Time over, thanks")
    # recoginize_() method will throw a request error if the API is unreachable, hence using exception handling
        try:
            # using google speech recognition
            text=r.recognize_google(audio_text, language = 'en')
            print(text)
            if text in classNames:
                scan="Scan the room"
                file = "scan.mp3"
                # initialize tts, create mp3 and play
                tts = gTTS(scan)
                tts.save(file)
                os.system("mpg123 " + file)
                playsound(file)
                return text
                
            else:
                err2='sorry I was not trained'
                file = "err1.mp3"
                # initialize tts, create mp3 and play
                tts = gTTS(err2)
                tts.save(file)
                os.system("mpg123 " + file)
                playsound(file)
        except:
                err2="sorry i did not get that"
                file = "err2.mp3"
                # initialize tts, create mp3 and play
                #tts = gTTS(err2)
                #tts.save(file)
                os.system("mpg123 " + file)
                playsound(file)
talk()

In [27]:
#Note if the voice voice to txt ain't woking propaly declear the var find manually i.e find='bottle'

find='bottle'   # or find = talk()
find_id=classNames[find]

In [32]:
left=True
down= True
up=True
right=True
cascPath = sys.argv[1]
faceCascade = cv2.CascadeClassifier(cascPath)
cap= cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame_resized = cv2.resize(frame,(300,300)) # resize frame for prediction

    # MobileNet requires fixed dimensions for input image(s)
    # so we have to ensure that it is resized to 300x300 pixels.
    # set a scale factor to image because network the objects has differents size. 
    # We perform a mean subtraction (127.5, 127.5, 127.5) to normalize the input;
    # after executing this command our "blob" now has the shape:
    # (1, 3, 300, 300)
    blob = cv2.dnn.blobFromImage(frame_resized, 0.007843, (300, 300), (127.5, 127.5, 127.5), False)
    #Set to network the input blob 
    net.setInput(blob)
    #Prediction of network
    detections = net.forward()

    #Size of frame resize (300x300)
    cols = frame_resized.shape[1] 
    rows = frame_resized.shape[0]

    #For get the class and location of object detected, 
    # There is a fix index for class, location and confidence
    # value in @detections array .

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2] #Confidence of prediction

        if confidence > 0.75: #Filter prediction 
            class_id = int(detections[0, 0, i, 1]) #Class label
            if  class_id == find_id:

                # Object location 
                xLeftBottom = int(detections[0, 0, i, 3] * cols) 
                yLeftBottom = int(detections[0, 0, i, 4] * rows)
                xRightTop   = int(detections[0, 0, i, 5] * cols)
                yRightTop   = int(detections[0, 0, i, 6] * rows)

                # Centre
                xcentre=int((xRightTop+xLeftBottom)/2) 
                ycentre=int((yRightTop+yLeftBottom)/2)

                # Draw location of central square
                x_b= int((cols/2)-20) 
                x_t= int((cols/2)+20)
                y_r= int((rows/2)+20)
                y_l= int((rows/2)-20)

                frame_resized= cv2.rectangle(frame_resized, (x_t, y_l), (x_b, y_r),(0, 255, 0))

                # Draw location of object
                frame_resized= cv2.rectangle(frame_resized, (xLeftBottom, yLeftBottom), (xRightTop, yRightTop),(0, 255, 0))

                # Direction
                if xcentre > x_t or xcentre < x_b:
                    if xcentre > x_t and right: 
                        right= False
                        left=True
                        up=True
                        down=True
                        print('go right')
                        break
    
                    if xcentre < x_b and left:
                        left= False
                        right=True
                        up=True
                        down=True
                        print('go left')
                        break
                    break

                if ycentre > y_r or ycentre < y_l:
                    if ycentre < y_l and up:
                        up= False
                        left=True
                        right=True
                        down=True
                        print('go up')
                        break
        
                    if ycentre > y_r and down:
                        down= False
                        up=True
                        left=True
                        right=True

            
                        print('go down')
                        break
                    break

                if ycentre < y_r and ycentre > y_l and  xcentre < x_t and xcentre > x_b:
                    #rest
                    left=True
                    right=True
                    up=True
                    down=True
                    print('stop')
                
                break #Print class and confidence
                               
    #cv2.namedWindow("frame", cv2.WINDOW_NORMAL)
    cv2.imshow("frame", frame_resized)
    if cv2.waitKey(1) >= 0:
        cap.release()
        cv2.destroyAllWindows()  # Break with ESC 
        break

go left
stop
stop
stop
go down
go left
go right
go down
go left
go up
go right
stop
stop
stop
stop
stop
stop
go left
stop
stop
stop
stop
go left
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
go left
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
stop
go down
go left
go right
go down
go right
